# IMPORT LIBRARIES

In [1]:
!pip install lightgbm
!pip install optuna

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, RobustScaler, StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

import lightgbm as lgb
import optuna

# GET THE DATA

In [3]:
train= pd.read_csv('dataset/train.csv')
test= pd.read_csv('dataset/test.csv')
test_ids= test['customer_id']

# PREPROCESS

In [4]:
train.loc[28498, 'gender']= 'M'

In [5]:
train[train['gender']== 'XNA']

,customer_id,name,age,gender,owns_car,owns_house,no_of_children,net_yearly_income,no_of_days_employed,occupation_type,total_family_members,migrant_worker,yearly_debt_payments,credit_limit,credit_limit_used(%),credit_score,prev_defaults,default_in_last_6months,credit_card_default


In [6]:
train[train['net_yearly_income']>= 1e8]

,customer_id,name,age,gender,owns_car,owns_house,no_of_children,net_yearly_income,no_of_days_employed,occupation_type,total_family_members,migrant_worker,yearly_debt_payments,credit_limit,credit_limit_used(%),credit_score,prev_defaults,default_in_last_6months,credit_card_default
26662,CST_114967,Jonathan Stempel,51,F,N,Y,1.0,1.407590e+08,924.0,Laborers,3.0,0.0,32825.6,31129970.49,82,526.0,0,1,1


In [7]:
train[train['credit_limit']>= 1e7]

,customer_id,name,age,gender,owns_car,owns_house,no_of_children,net_yearly_income,no_of_days_employed,occupation_type,total_family_members,migrant_worker,yearly_debt_payments,credit_limit,credit_limit_used(%),credit_score,prev_defaults,default_in_last_6months,credit_card_default
26662,CST_114967,Jonathan Stempel,51,F,N,Y,1.0,1.407590e+08,924.0,Laborers,3.0,0.0,32825.6,31129970.49,82,526.0,0,1,1


In [8]:
train= train.drop(26662, axis= 0)

In [9]:
t= train['no_of_days_employed']

In [10]:
u= t.loc[t < 300000]

In [11]:
train.loc[train['no_of_days_employed']> 300000, 'no_of_days_employed']= u.mean()

In [12]:
categorical_features= ['gender', 'owns_car', 'owns_house', 'occupation_type']

In [13]:
for i in categorical_features:
    df1= pd.get_dummies(train[i], prefix= '{}'.format(i))
    train= pd.concat([train, df1], axis= 1)
    train= train.drop([i], axis= 1)

In [14]:
train= train.drop(['gender_M', 'owns_car_N', 'owns_house_N'], axis= 1)

In [15]:
test.loc[3510, 'gender']= 'F'

In [16]:
for i in categorical_features:
    df1= pd.get_dummies(test[i], prefix= '{}'.format(i))
    test= pd.concat([test, df1], axis= 1)
    test= test.drop([i], axis= 1)
    
test= test.drop(['gender_M', 'owns_car_N', 'owns_house_N'], axis= 1)

In [17]:
y= train['credit_card_default']
X= train.drop(['customer_id', 'name', 'credit_card_default'], axis= 1)
test= test.drop(['customer_id', 'name'], axis= 1)

In [18]:
scaler= RobustScaler()
scaler.fit(X)
X= scaler.transform(X)
test= scaler.transform(test)

# TRAIN AND MAKE PREDS

In [19]:
oof_preds= np.empty(train.shape[0])
test_preds= np.empty((test.shape[0], 10))

def hyper_tune(trial, data= X, target= y, oof_preds= oof_preds, test_preds= test_preds):
    kfold= StratifiedKFold(n_splits= 10, shuffle= True, random_state= 2021)
    
    params= {'objective': 'binary',
             'metric': 'binary_logloss',
             'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
             'learning_rate': trial.suggest_float('learning_rate', 0.001, 1),
             'num_leaves': trial.suggest_int('num_leaves', 30, 3000),
             'max_depth': trial.suggest_int('max_depth', 3, 50),
             "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
             "max_bin": trial.suggest_int("max_bin", 200, 300),
             "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
             "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
             "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
             "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 0.95, step=0.1),
             "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
             "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 0.95, step=0.1),
             'random_state': 2021
            }
    
    for fold, (train_idx, val_idx) in enumerate(kfold.split(X, y)):
        X_train, X_val= X[train_idx], X[val_idx]
        y_train, y_val= y.iloc[train_idx], y.iloc[val_idx]
        model= lgb.LGBMClassifier(**params)
        model.fit(X_train, 
                  y_train,
                  eval_set= [(X_val, y_val)],
                  eval_metric= 'binary_logloss',
                  callbacks= [lgb.early_stopping(stopping_rounds= trial.suggest_int('stopping_rounds', 20, 200)),
                             lgb.log_evaluation(period= 50)])
        
        oof_preds[val_idx]= model.predict(X_val)
        test_preds[:, fold]= model.predict(test)
        
    score= 100 * f1_score(y, oof_preds, average= 'macro')
    return score

In [20]:
study= optuna.create_study(direction= 'maximize')
study.optimize(hyper_tune, n_trials= 10)

[I 2021-12-25 23:20:27,630] A new study created in memory with name: no-name-4b500cbd-29ca-4ed5-9a2d-0a732efd6b1f
C:\Users\SAGRIKA\anaconda3\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(


[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=7300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7300
[LightGBM] [Warning] min_gain_to_split is set=5.688896301002334, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=5.688896301002334
[LightGBM] [Warning] lambda_l1 is set=80, reg_alpha=0.0 will be ignored. Current value: lambda_l1=80
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] lambda_l2 is set=5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=5
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
Training until validation scores don't improve for 93 rounds
[50]	valid_0's binary_logloss: 0.07837
[100]	valid_0's binary_logloss: 0.0782639
[150]	valid_0's binary_logloss

[50]	valid_0's binary_logloss: 0.0783114
[100]	valid_0's binary_logloss: 0.0777527
[150]	valid_0's binary_logloss: 0.0777527
Early stopping, best iteration is:
[95]	valid_0's binary_logloss: 0.0777527
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=7300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7300
[LightGBM] [Warning] min_gain_to_split is set=5.688896301002334, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=5.688896301002334
[LightGBM] [Warning] lambda_l1 is set=80, reg_alpha=0.0 will be ignored. Current value: lambda_l1=80
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] lambda_l2 is set=5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=5
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. C

[I 2021-12-25 23:20:32,895] Trial 0 finished with value: 91.45246886538582 and parameters: {'n_estimators': 703, 'learning_rate': 0.19305426632035, 'num_leaves': 1896, 'max_depth': 32, 'min_data_in_leaf': 7300, 'max_bin': 285, 'lambda_l1': 80, 'lambda_l2': 5, 'min_gain_to_split': 5.688896301002334, 'bagging_fraction': 0.5, 'bagging_freq': 1, 'feature_fraction': 0.8, 'stopping_rounds': 93}. Best is trial 0 with value: 91.45246886538582.


[200]	valid_0's binary_logloss: 0.0720401
[250]	valid_0's binary_logloss: 0.0720401
Early stopping, best iteration is:
[184]	valid_0's binary_logloss: 0.0720401


...

[I 2021-12-25 23:21:10,738] Trial 8 finished with value: 92.48844494674942 and parameters: {'n_estimators': 339, 'learning_rate': 0.04682638928330207, 'num_leaves': 2185, 'max_depth': 23, 'min_data_in_leaf': 500, 'max_bin': 287, 'lambda_l1': 95, 'lambda_l2': 80, 'min_gain_to_split': 5.351756498903151, 'bagging_fraction': 0.30000000000000004, 'bagging_freq': 1, 'feature_fraction': 0.5, 'stopping_rounds': 78}. Best is trial 8 with value: 92.48844494674942.
C:\Users\SAGRIKA\anaconda3\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(


[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=7200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7200
[LightGBM] [Warning] min_gain_to_split is set=11.986863146721895, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=11.986863146721895
[LightGBM] [Warning] lambda_l1 is set=85, reg_alpha=0.0 will be ignored. Current value: lambda_l1=85
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] lambda_l2 is set=40, reg_lambda=0.0 will be ignored. Current value: lambda_l2=40
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
Training until validation scores don't improve for 24 rounds
[50]	valid_0's binary_logloss: 0.149074
[100]	valid_0's binary_logloss: 0.102668
[150]	valid_0's binary_log

[50]	valid_0's binary_logloss: 0.145369
[100]	valid_0's binary_logloss: 0.0966722
[150]	valid_0's binary_logloss: 0.0786343
[200]	valid_0's binary_logloss: 0.0732854
[250]	valid_0's binary_logloss: 0.0718217
Early stopping, best iteration is:
[249]	valid_0's binary_logloss: 0.0718217
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=7200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7200
[LightGBM] [Warning] min_gain_to_split is set=11.986863146721895, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=11.986863146721895
[LightGBM] [Warning] lambda_l1 is set=85, reg_alpha=0.0 will be ignored. Current value: lambda_l1=85
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] lambda_l2 is set=40, reg_lambda=0.0 will be ignored. Current value: lamb

[I 2021-12-25 23:21:18,464] Trial 9 finished with value: 91.76796362840845 and parameters: {'n_estimators': 851, 'learning_rate': 0.029841058300664556, 'num_leaves': 2903, 'max_depth': 45, 'min_data_in_leaf': 7200, 'max_bin': 232, 'lambda_l1': 85, 'lambda_l2': 40, 'min_gain_to_split': 11.986863146721895, 'bagging_fraction': 0.5, 'bagging_freq': 1, 'feature_fraction': 0.8, 'stopping_rounds': 24}. Best is trial 8 with value: 92.48844494674942.


[250]	valid_0's binary_logloss: 0.0763002
Early stopping, best iteration is:
[255]	valid_0's binary_logloss: 0.0762609


In [21]:
study.best_trial.params

{'n_estimators': 339,
 'learning_rate': 0.04682638928330207,
 'num_leaves': 2185,
 'max_depth': 23,
 'min_data_in_leaf': 500,
 'max_bin': 287,
 'lambda_l1': 95,
 'lambda_l2': 80,
 'min_gain_to_split': 5.351756498903151,
 'bagging_fraction': 0.30000000000000004,
 'bagging_freq': 1,
 'feature_fraction': 0.5,
 'stopping_rounds': 78}

In [22]:
study.best_value

92.48844494674942

In [23]:
test_preds= pd.DataFrame(test_preds).mode(axis= 1)[0]

In [24]:
sub= pd.DataFrame({'customer_id': test_ids, 'credit_card_default': test_preds})
sub.to_csv('submission_lgbm.csv', index= False)